In [4]:
pip install scipy

  Obtaining dependency information for scipy from https://files.pythonhosted.org/packages/06/15/e73734f9170b66c6a84a0bd7e03586e87e77404e2eb8e34749fc49fa43f7/scipy-1.11.2-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/59.1 kB ? eta -:--:--
     ------ --------------------------------- 10.2/59.1 kB ? eta -:--:--
     ------------------- ------------------ 30.7/59.1 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 59.1/59.1 kB 390.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/44.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/44.0 MB 1.7 MB/s eta 0:00:27
   ---------------------------------------- 0.1/44.0 MB 1.8 MB/s eta 0:00:25
   ---------------------------------------- 0.3/44.0 MB 2.1 MB/s eta 0:00:21
   ---------------------------------------- 0.5/44.0 MB 3.0 MB/s eta 0:00:15
    --------------------------------------- 0.8/44.0 MB 3.7 MB/s eta 0:00:12
   - ------------------------------

In [5]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [6]:
dict_table = pickle.load(open('dict_table','rb'))
df_historical_data = pd.read_csv('clean_fifa_worldcup_matches.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')

In [7]:
df_home = df_historical_data[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_historical_data[['AwayTeam', 'HomeGoals', 'AwayGoals']]

df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

df_team_strength = pd.concat([df_home, df_away], ignore_index=True).groupby(['Team']).mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,0.863636,1.409091
Angola,0.333333,0.666667
Argentina,1.741379,1.068966
Australia,0.840000,1.880000
Austria,1.348837,1.511628
...,...,...
Uruguay,1.408451,1.309859
Wales,0.800000,0.800000
West Germany,2.035294,1.141176


In [8]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

In [9]:
print(predict_points('England', 'United States'))
print(predict_points('Argentina', 'Mexico'))
print(predict_points('Qatar (H)', 'Ecuador')) # Qatar vs Team X -> 0 points to both

(2.24003239704936, 0.5755072718935664)
(2.2397992412507435, 0.598823312417191)
(0, 0)


In [10]:
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [12]:
df_fixture_group_48

,home,score,away,year
0,Qatar,0–2,Ecuador,2022
1,Senegal,0–2,Netherlands,2022
2,Qatar,1–3,Senegal,2022
3,Netherlands,1–1,Ecuador,2022
4,Ecuador,1–2,Senegal,2022
5,Netherlands,2–0,Qatar,2022
6,England,6–2,Iran,2022
7,United States,1–1,Wales,2022
8,Wales,0–2,Iran,2022
9,England,0–0,United States,2022


In [15]:
for group in dict_table:
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [16]:
dict_table['Group A']

,Team,Pts
0,Netherlands,15.0
1,Senegal,10.0
2,Ecuador,8.0
3,Qatar (H),0.0


In [17]:
df_fixture_knockout

,home,score,away,year
48,Netherlands,3–1,United States,2022
49,Argentina,2–1,Australia,2022
50,France,3–1,Poland,2022
51,England,3–0,Senegal,2022
52,Japan,1–1 (a.e.t.),Croatia,2022
53,Brazil,4–1,South Korea,2022
54,Morocco,0–0 (a.e.t.),Spain,2022
55,Portugal,6–1,Switzerland,2022


In [18]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,home,score,away,year,winner
48,Netherlands,3–1,United States,2022,?
49,Argentina,2–1,Australia,2022,?
50,France,3–1,Poland,2022,?
51,England,3–0,Senegal,2022,?
52,Japan,1–1 (a.e.t.),Croatia,2022,?
53,Brazil,4–1,South Korea,2022,?
54,Morocco,0–0 (a.e.t.),Spain,2022,?
55,Portugal,6–1,Switzerland,2022,?


In [19]:
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [20]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,3–1,United States,2022,Netherlands
49,Argentina,2–1,Australia,2022,Argentina
50,France,3–1,Poland,2022,France
51,England,3–0,Senegal,2022,England
52,Japan,1–1 (a.e.t.),Croatia,2022,Croatia
53,Brazil,4–1,South Korea,2022,Brazil
54,Morocco,0–0 (a.e.t.),Spain,2022,Spain
55,Portugal,6–1,Switzerland,2022,Portugal


In [21]:
def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [22]:
update_table(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Croatia,1–1 (a.e.t.),Brazil,2022,?
57,Netherlands,2–2 (a.e.t.),Argentina,2022,?
58,Morocco,1–0,Portugal,2022,?
59,England,1–2,France,2022,?


In [23]:
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Croatia,1–1 (a.e.t.),Brazil,2022,Brazil
57,Netherlands,2–2 (a.e.t.),Argentina,2022,Netherlands
58,Morocco,1–0,Portugal,2022,Portugal
59,England,1–2,France,2022,England


In [24]:
update_table(df_fixture_quarter, df_fixture_semi)

,home,score,away,year,winner
60,Argentina,3–0,Croatia,2022,?
61,France,2–0,Morocco,2022,?


In [25]:
get_winner(df_fixture_semi)

,home,score,away,year,winner
60,Argentina,3–0,Croatia,2022,Argentina
61,France,2–0,Morocco,2022,France


In [26]:
update_table(df_fixture_semi, df_fixture_final)


,home,score,away,year,winner
62,Croatia,2–1,Morocco,2022,?
63,Argentina,3–3 (a.e.t.),France,2022,?


In [27]:
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Croatia,2–1,Morocco,2022,Croatia
63,Argentina,3–3 (a.e.t.),France,2022,Argentina
